In [ ]:
'''
Data can be accessed 
from the following link:   
https://data.humdata.org/dataset?dataseries_name=Data+for+Good+at+Meta+-+High+Resolution+Population+Density+Maps+and+Demographic+Estimates
'''

# Access to improved water sources with school and hospital overlay and built-up underlay

In [18]:
# package import
import pandas as pd
import geopandas as gpd
import os 
import osmnx as ox
import xarray as xr
import rioxarray as rxr

os.chdir(r'C:/Users/jtrum/world_bank/data/')

In [11]:
# read in improved water sources data
water = xr.open_rasterio('IHME_LMIC_WASH_2000_2017_W_IMP_PERCENT_MEAN_2017_Y2020M06D02.TIF')

# read in built-up data raster
built = xr.open_rasterio('WSF2019_v1_12_-10.tif')

# read in AOI
aoi = gpd.read_file('aoiLuanda.geojson')

# clip raster to AOI
clip = aoi.geometry
water_clip = water.rio.clip(clip, from_disk=True)
built_clip = built.rio.clip(clip, from_disk=True)

In [26]:
built = xr.open_dataset('WSF2019_v1_12_-10.tif')
built_clip = built.rio.clip(clip, from_disk=True)
built_gdf = built_clip.to_dataframe().reset_index()


In [35]:
# drop columns 'band' and 'spatial_ref'
built_gdf = built_gdf.drop(['band', 'spatial_ref'], axis=1)
# rename 'band_data' to 'value'
built_gdf = built_gdf.rename(columns={'band_data': 'value'})
# drop NaN values
built_gdf = built_gdf.dropna()
# convert 'x' and 'y' columns to geometry
built_gdf = gpd.GeoDataFrame(built_gdf, geometry=gpd.points_from_xy(built_gdf.x, built_gdf.y))
# set CRS to WGS84
built_gdf.crs = 'EPSG:4326'
# drop 'x' and 'y' columns
built_gdf = built_gdf.drop(['x', 'y'], axis=1)
# rename 0 values in 'value' to 'unbuilt' and 255 values to 'built'

built_gdf

KeyError: "['band', 'spatial_ref'] not found in axis"

In [36]:
built_gdf['value'] = built_gdf['value'].replace({0: 'unbuilt', 255: 'built'})

In [40]:
from sklearn.cluster import DBSCAN

In [44]:
cluster = DBSCAN(eps=0.1, min_samples=5).fit(built_gdf.geometry).fit(built_gdf.geometry.values.reshape(-1, 1))
built_gdf['cluster'] = cluster.labels_
polygons = built_gdf.dissolve(by='cluster')

In [37]:
built_gdf

,value,geometry
4814,unbuilt,POINT (12.99256 -9.06890)
4815,unbuilt,POINT (12.99256 -9.06899)
4816,unbuilt,POINT (12.99256 -9.06908)
4817,unbuilt,POINT (12.99256 -9.06917)
4818,unbuilt,POINT (12.99256 -9.06926)
...,...,...
56257733,unbuilt,POINT (13.63387 -8.85321)
56257734,unbuilt,POINT (13.63387 -8.85330)
56265612,unbuilt,POINT (13.63396 -8.85312)
56265613,unbuilt,POINT (13.63396 -8.85321)


In [14]:
# binarize built clip so that 0's = not developed and 255's = developed
# built_clip = built_clip.where(built_clip == 255, 'developed')
# built_clip = built_clip.where(built_clip == 0, 'undeveloped')

# convert built_clip to geodataframe
built_gdf = built_clip.to_gdf()
# built_clip = built_clip.reset_index()
# built_clip = built_clip.drop(columns=['band'])
# built_clip = gpd.GeoDataFrame(built_clip, geometry=gpd.points_from_xy(built_clip.x, built_clip.y))
# built_clip = built_clip.drop(columns=['x', 'y'])
# built_clip = built_clip.set_crs('EPSG:4326')
built_clip

AttributeError: 'DataArray' object has no attribute 'to_gdf'